In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
                              # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)

In [170]:
base_url = 'http://www.flyday.co.kr'
#li == $0으로 묶여있기 때문에 일단 해결방법을 찾지 못하여 href로 묶여있는 카테고리별 전체상품의 url을 직접 긁음
category_list = ['/shop/shopbrand.html?xcode=018&type=X','/shop/shopbrand.html?xcode=005&type=X','/shop/shopbrand.html?xcode=017&type=X','/shop/shopbrand.html?xcode=019&type=X','/shop/shopbrand.html?xcode=074&type=X']

category_list_url = [] # 각 카테고리별 접근 주소 저장
for i in range(len(category_list)) : # base_url과 각 카테고리의 href를 합쳐야 해당 카테고리 주소로 접근할 수 있으므로 합쳐주는 작업을 진행하는 반복문
    url = base_url + category_list[i]
    category_list_url.append(url)

In [171]:
category_list_url

['http://www.flyday.co.kr/shop/shopbrand.html?xcode=018&type=X',
 'http://www.flyday.co.kr/shop/shopbrand.html?xcode=005&type=X',
 'http://www.flyday.co.kr/shop/shopbrand.html?xcode=017&type=X',
 'http://www.flyday.co.kr/shop/shopbrand.html?xcode=019&type=X',
 'http://www.flyday.co.kr/shop/shopbrand.html?xcode=074&type=X']

# TEE

**각 상품별 클래스 획득**

# 위에 까지는 다맞음 근데 사이즈표 추출이 문제임... -> 일단 이미지만 추출

**url에서 필요한 정보를 추출하기 위해 소스작업**

In [ ]:
#상품별 사이즈가 표로 되어있는 것이 아니라 p태그로 class = txt_2로 감싸어져 있음

In [155]:
list = pd.DataFrame({'color_view' : []})
base_url = 'http://www.flyday.co.kr'
#li == $0으로 묶여있기 때문에 일단 해결방법을 찾지 못하여 href로 묶여있는 카테고리별 전체상품의 url을 직접 긁음
category_list = ['/shop/shopbrand.html?xcode=018&type=X','/shop/shopbrand.html?xcode=005&type=X','/shop/shopbrand.html?xcode=017&type=X','/shop/shopbrand.html?xcode=019&type=X','/shop/shopbrand.html?xcode=074&type=X']

category_list_url = [] # 각 카테고리별 접근 주소 저장
for i in range(len(category_list)) : # base_url과 각 카테고리의 href를 합쳐야 해당 카테고리 주소로 접근할 수 있으므로 합쳐주는 작업을 진행하는 반복문
    url = base_url + category_list[i]
    category_list_url.append(url)



# 플라이데이는 페이지수가 넘어 갈수록 끝에 &page=1 , &page=2 , &page=3 형식으로 일정한 패턴을 가진다.

for i in range(len(category_list_url)):
    start_url = category_list_url[i]
    start_url_name = category_list_url[i]
    page_url = category_list_url[i] + '&page=' + '{}'

    page1 = requests.get(page_url)
    page2 = BeautifulSoup(page1.content,'html.parser') #홈페이지 html획득
    page3 = page2.find('div',{'class':'item-page'})
    page4 = page3.find_all('a')
    page5 = len(page4)-1#마지막 페이지와 함께 그 옆에 끝이라는 단어가 있는데 해당 단어 또한 같은 태그로 묶여 있기 때문에 -1을 해줌

    for page in range(1,page5+1) : # 각 페이지를 반복해서 데이터를 수집하기 위한 반복문
        res = requests.get(page_url.format(str(page)))
        html = BeautifulSoup(res.content, 'html.parser') # html 소스 획득
        table = html.find_all('dt', { 'class': 'thumb' }) # 각각의 상품에 접근하기 위해 클래스획득
    
    
    #category href가져온 것처럼 상품별 href가져오기
        product = []
        for i in range(len(table)):
            pro = table[i].find('a')['href']
            product.append(pro)

        pro_url_list = []
        for i in range(len(product)):
            pro_url = base_url + product[i]
            pro_url_list.append(pro_url)

        for i in pro_url_list:
            res = requests.get(i)
            html = BeautifulSoup(res.content,'html.parser')
            table = html.find('div',{'class','page-body'})
            img = table.find_all('img')

            src = []
            for i in range(len(img)):
                img_src = img[i]['src']
                src.append(img_src)

            cv = base_url + src[0]
            df = pd.DataFrame({'color_view' :[cv]})
            list = list.append(df) # 데이터 프레임 형식으로 저장       

In [156]:
list.index = range(len(list)) 

In [157]:
list

,color_view
0,http://www.flyday.co.kr/shopimages/flyday/0180...
1,http://www.flyday.co.kr/shopimages/flyday/0180...
2,http://www.flyday.co.kr/shopimages/flyday/0180...
3,http://www.flyday.co.kr/shopimages/flyday/0180...
4,http://www.flyday.co.kr/shopimages/flyday/0180...
5,http://www.flyday.co.kr/shopimages/flyday/0180...
6,http://www.flyday.co.kr/shopimages/flyday/0180...
7,http://www.flyday.co.kr/shopimages/flyday/0180...
8,http://www.flyday.co.kr/shopimages/flyday/0180...
9,http://www.flyday.co.kr/shopimages/flyday/0180...


In [158]:
import urllib.request # 이미지 파일을 저장하기 위한 패키지, 웹사이트의 이미지 주소가 있으면 다운 받을 수 있다.

In [ ]:
for i in range(len(list)) : # 옷 이미지에 대한 이미지 파일을 저장하는 것, 반복문을 통해 하나씩 저장해주었다.
    url = list['color_view'].iloc[i]
    name = category_list_name[0] + 'color_view' + str(i) + '.jpg' # ex) 아우터color_view100.jpg 형식으로 파일을 저장하였다.
    urllib.request.urlretrieve(url, name)

# 지그재그형식갖춰서 크롤링

In [ ]:
#티의 url을 알고 있으니 그것 부터 시작하면 됨
#category_list_url[0] <- 티의 url임.

#그다음부턴 bs4 parser